 1. Librerías y Configuración Inicial


In [302]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt


2. Carga de Datos y Exploración Preliminar

In [303]:
# Carga de datos
users = pd.read_csv("Final_Updated_Expanded_Users.csv")
destinations = pd.read_csv("Expanded_Destinations.csv")
history = pd.read_csv("Final_Updated_Expanded_UserHistory.csv")
reviews = pd.read_csv("Final_Updated_Expanded_Reviews.csv")

# Tipado uniforme
for df in [users, history, reviews]:
    df['UserID'] = df['UserID'].astype(str)
for df in [destinations, history, reviews]:
    df['DestinationID'] = df['DestinationID'].astype(str)

# Validación y exploración
print(f"Usuarios: {users.shape}")
print(f"Destinos: {destinations.shape}")
print(f"Historial: {history.shape}")
print(f"Reviews: {reviews.shape}")

print(f"  - En Users: {users['UserID'].nunique()}")
print(f"  - En History: {history['UserID'].nunique()}")
print(f"  - En Reviews: {reviews['UserID'].nunique()}")
print(f"  - En Destinations: {destinations['DestinationID'].nunique()}")
print(f"  - En History: {history['DestinationID'].nunique()}")
print(f"  - En Reviews: {reviews['DestinationID'].nunique()}")

print("Users:", users.isnull().sum().to_dict())
print("History:", history.isnull().sum().to_dict())
print("Reviews:", reviews.isnull().sum().to_dict())
print("Destinations:", destinations.isnull().sum().to_dict())

# Duplicados
print(f"  - Users: {users.duplicated().sum()}")
print(f"  - History: {history.duplicated().sum()}")
print(f"  - Reviews: {reviews.duplicated().sum()}")
print(f"  - Destinations: {destinations.duplicated().sum()}")

# Longitud de las reviews
reviews['ReviewLength'] = reviews['ReviewText'].astype(str).apply(len)
print("\nRevisión de reviews textuales:")
print(" - Vacías o nulas:", reviews['ReviewText'].isnull().sum())
print(" - Muy cortas (<10):", (reviews['ReviewLength'] < 10).sum())
print(" - Longitud promedio:", reviews['ReviewLength'].mean())


Usuarios: (999, 7)
Destinos: (1000, 6)
Historial: (999, 5)
Reviews: (999, 5)
  - En Users: 999
  - En History: 642
  - En Reviews: 624
  - En Destinations: 1000
  - En History: 638
  - En Reviews: 651
Users: {'UserID': 0, 'Name': 0, 'Email': 0, 'Preferences': 0, 'Gender': 0, 'NumberOfAdults': 0, 'NumberOfChildren': 0}
History: {'HistoryID': 0, 'UserID': 0, 'DestinationID': 0, 'VisitDate': 0, 'ExperienceRating': 0}
Reviews: {'ReviewID': 0, 'DestinationID': 0, 'UserID': 0, 'Rating': 0, 'ReviewText': 0}
Destinations: {'DestinationID': 0, 'Name': 0, 'State': 0, 'Type': 0, 'Popularity': 0, 'BestTimeToVisit': 0}
  - Users: 0
  - History: 0
  - Reviews: 0
  - Destinations: 0

Revisión de reviews textuales:
 - Vacías o nulas: 0
 - Muy cortas (<10): 0
 - Longitud promedio: 19.0


 3. Unión de Datasets (Merge)

In [304]:
import pandas as pd

# Cargar los archivos
users_df = pd.read_csv("Final_Updated_Expanded_Users.csv")
destinations_df = pd.read_csv("Expanded_Destinations.csv")
user_history_df = pd.read_csv("Final_Updated_Expanded_UserHistory.csv")
reviews_df = pd.read_csv("Final_Updated_Expanded_Reviews.csv")

# Paso 1: Unir historial con usuarios
merged_df = pd.merge(user_history_df, users_df, on="UserID", how="left")

# Paso 2: Unir con destinos
merged_df = pd.merge(merged_df, destinations_df, on="DestinationID", how="left")


# Guardar el resultado
merged_df.to_csv("Merged_Travel_Data.csv", index=False)

print("Archivo unido y guardado como 'Merged_Travel_Data.csv'")
m = pd.read_csv("Merged_Travel_Data.csv")
m.head()


Archivo unido y guardado como 'Merged_Travel_Data.csv'


,HistoryID,UserID,DestinationID,VisitDate,ExperienceRating,Name_x,Email,Preferences,Gender,NumberOfAdults,NumberOfChildren,Name_y,State,Type,Popularity,BestTimeToVisit
0,1,525,760,2024-01-01,3,Saanvi,saanvi@example.com,"City, Historical",Female,2,2,Leh Ladakh,Jammu and Kashmir,Adventure,8.352180,Apr-Jun
1,2,184,532,2024-02-15,5,Ishaan,ishaan@example.com,"Beaches, Historical",Male,1,2,Goa Beaches,Goa,Beach,8.988127,Nov-Mar
2,3,897,786,2024-03-20,2,Pooja,pooja@example.com,"City, Historical",Female,1,2,Taj Mahal,Uttar Pradesh,Historical,8.389206,Nov-Feb
3,4,470,660,2024-01-01,1,Arjun,arjun@example.com,"Nature, Adventure",Male,2,1,Leh Ladakh,Jammu and Kashmir,Adventure,7.923388,Apr-Jun
4,5,989,389,2024-02-15,4,Ritvik,ritvik@example.com,"Nature, Adventure",Male,2,1,Kerala Backwaters,Kerala,Nature,9.409146,Sep-Mar


In [305]:
# Cargar datos
df = pd.read_csv("Merged_Travel_Data.csv")

print("Shape:", df.shape)
print("Columnas:", df.columns.tolist())
print("Valores nulos por columna:\n", df.isnull().sum())




Shape: (999, 16)
Columnas: ['HistoryID', 'UserID', 'DestinationID', 'VisitDate', 'ExperienceRating', 'Name_x', 'Email', 'Preferences', 'Gender', 'NumberOfAdults', 'NumberOfChildren', 'Name_y', 'State', 'Type', 'Popularity', 'BestTimeToVisit']
Valores nulos por columna:
 HistoryID           0
UserID              0
DestinationID       0
VisitDate           0
ExperienceRating    0
Name_x              0
Email               0
Preferences         0
Gender              0
NumberOfAdults      0
NumberOfChildren    0
Name_y              0
State               0
Type                0
Popularity          0
BestTimeToVisit     0
dtype: int64


In [306]:
# 4) Eliminar columnas irrelevantes
df_clean = df.drop(columns=["HistoryID", "Name_x", "Email"])

# 5) Convertir fechas a datetime
df_clean["VisitDate"] = pd.to_datetime(df_clean["VisitDate"])


In [307]:
# 6) Crear columna PreferencesList (lista de etiquetas)
df_clean["PreferencesList"] = df_clean["Preferences"].str.split(", ")


In [308]:
# 7) Revisar duplicados completos
cols_without_list = [c for c in df_clean.columns if c != "PreferencesList"]

duplicates_full = df_clean[df_clean.duplicated(subset=cols_without_list)]
print("Duplicados completos encontrados:", len(duplicates_full))

# 8) Eliminar duplicados completos si los hay
df_clean = df_clean.drop_duplicates(subset=cols_without_list)


Duplicados completos encontrados: 0


In [309]:
# 9) Duplicados por combinación clave
duplicates_keys = df_clean[df_clean.duplicated(subset=["UserID", "DestinationID", "VisitDate"])]
print("Duplicados por UserID + DestinationID + VisitDate encontrados:", len(duplicates_keys))

# 10) Eliminar si hubiera
df_clean = df_clean.drop_duplicates(subset=["UserID", "DestinationID", "VisitDate"])


Duplicados por UserID + DestinationID + VisitDate encontrados: 0


In [310]:
# 11) Ver etiquetas antes de limpiar
print("Unique 'Type':", df_clean["Type"].unique())
print("Unique 'Preferences':", df_clean["Preferences"].unique())
print("Unique 'BestTimeToVisit':", df_clean["BestTimeToVisit"].unique())


Unique 'Type': ['Adventure' 'Beach' 'Historical' 'Nature' 'City']
Unique 'Preferences': ['City, Historical' 'Beaches, Historical' 'Nature, Adventure']
Unique 'BestTimeToVisit': ['Apr-Jun' 'Nov-Mar' 'Nov-Feb' 'Sep-Mar' 'Oct-Mar']


In [311]:
# 12) Normalizar columnas categóricas
df_clean["Type"] = df_clean["Type"].str.strip().str.title()
df_clean["Preferences"] = df_clean["Preferences"].str.strip().str.title()
df_clean["BestTimeToVisit"] = df_clean["BestTimeToVisit"].str.strip().str.title()

# 13) Verificar de nuevo
print("Unique 'Type' después de normalizar:", df_clean["Type"].unique())
print("Unique 'Preferences' después de normalizar:", df_clean["Preferences"].unique())
print("Unique 'BestTimeToVisit' después de normalizar:", df_clean["BestTimeToVisit"].unique())


Unique 'Type' después de normalizar: ['Adventure' 'Beach' 'Historical' 'Nature' 'City']
Unique 'Preferences' después de normalizar: ['City, Historical' 'Beaches, Historical' 'Nature, Adventure']
Unique 'BestTimeToVisit' después de normalizar: ['Apr-Jun' 'Nov-Mar' 'Nov-Feb' 'Sep-Mar' 'Oct-Mar']


In [312]:
# 14) Revisión de nulos
print("Valores nulos por columna:\n", df_clean.isnull().sum())


Valores nulos por columna:
 UserID              0
DestinationID       0
VisitDate           0
ExperienceRating    0
Preferences         0
Gender              0
NumberOfAdults      0
NumberOfChildren    0
Name_y              0
State               0
Type                0
Popularity          0
BestTimeToVisit     0
PreferencesList     0
dtype: int64


In [313]:
# Revisar distribución de variables categóricas
print("\nDistribución 'Type':")
print(df_clean["Type"].value_counts())

print("\nDistribución 'Preferences':")
print(df_clean["Preferences"].value_counts())

print("\nDistribución 'Gender':")
print(df_clean["Gender"].value_counts())

print("\nDistribución 'BestTimeToVisit':")
print(df_clean["BestTimeToVisit"].value_counts())

# Revisión de variables numéricas
print("\nDistribución 'NumberOfAdults':")
print(df_clean["NumberOfAdults"].value_counts())

print("\nDistribución 'NumberOfChildren':")
print(df_clean["NumberOfChildren"].value_counts())

print("\nDistribución 'Popularity':")
print(df_clean["Popularity"].describe())

print("\nDistribución 'ExperienceRating':")
print(df_clean["ExperienceRating"].describe())



Distribución 'Type':
Type
Nature        211
Historical    209
Adventure     204
Beach         190
City          185
Name: count, dtype: int64

Distribución 'Preferences':
Preferences
City, Historical       339
Beaches, Historical    330
Nature, Adventure      330
Name: count, dtype: int64

Distribución 'Gender':
Gender
Female    514
Male      485
Name: count, dtype: int64

Distribución 'BestTimeToVisit':
BestTimeToVisit
Sep-Mar    211
Nov-Feb    209
Apr-Jun    204
Nov-Mar    190
Oct-Mar    185
Name: count, dtype: int64

Distribución 'NumberOfAdults':
NumberOfAdults
2    531
1    468
Name: count, dtype: int64

Distribución 'NumberOfChildren':
NumberOfChildren
0    344
2    333
1    322
Name: count, dtype: int64

Distribución 'Popularity':
count    999.000000
mean       8.537242
std        0.581242
min        7.504500
25%        8.037513
50%        8.501225
75%        9.080623
max        9.499811
Name: Popularity, dtype: float64

Distribución 'ExperienceRating':
count    999.000000
mean

In [314]:
df_clean["PopularityNorm"] = (df_clean["Popularity"] - df_clean["Popularity"].min()) / (df_clean["Popularity"].max() - df_clean["Popularity"].min())


In [315]:
df_clean["ExperienceRatingNorm"] = (df_clean["ExperienceRating"] - 1) / (5 - 1)


In [316]:
#guardar el DataFrame limpio
df_clean.to_csv("Cleaned_Travel_Data.csv", index=False)

In [317]:
user_item_matrix = df_clean.pivot_table(
    index="UserID",
    columns="DestinationID",
    values="ExperienceRatingNorm",
    fill_value=0
)


In [318]:
svd = TruncatedSVD(n_components=20, random_state=42)
item_embeddings = svd.fit_transform(user_item_matrix.T)
destination_similarity = cosine_similarity(item_embeddings)

# Índices de destinos
destination_indices = {
    dest: idx for idx, dest in enumerate(user_item_matrix.columns)
}


In [319]:
# Base para contenido
destination_content = df_clean[[
    "DestinationID",
    "Name_y",
    "PopularityNorm",
    "PreferencesList",
    "Type",
    "BestTimeToVisit"
]].drop_duplicates(subset=["DestinationID"])

# MultiLabelBinarizer para PreferencesList
mlb = MultiLabelBinarizer()
prefs_encoded = pd.DataFrame(
    mlb.fit_transform(destination_content["PreferencesList"]),
    columns=[f"Pref_{c}" for c in mlb.classes_]
)
prefs_encoded.index = destination_content.index

# One-hot encode Type y BestTimeToVisit
dummies = pd.get_dummies(destination_content[["Type", "BestTimeToVisit"]])

# Concatenar
destination_content_encoded = pd.concat([
    destination_content[["DestinationID", "Name_y", "PopularityNorm"]],
    dummies,
    prefs_encoded
], axis=1)

# Asegurarse que todo sea float
for col in destination_content_encoded.columns:
    if destination_content_encoded[col].dtype in [bool, int]:
        destination_content_encoded[col] = destination_content_encoded[col].astype(float)


In [320]:
content_similarity = cosine_similarity(
    destination_content_encoded.drop(columns=["DestinationID", "Name_y"])
)


In [321]:
def recommend_by_profile(user_profile, top_n=5):
    # One-hot de Type (puedes cambiar la lógica si quieres)
    type_cols = [c for c in destination_content_encoded.columns if c.startswith("Type_")]
    type_vec = pd.Series(0.0, index=type_cols)

    # One-hot de BestTimeToVisit (puedes ajustar la lógica)
    time_cols = [c for c in destination_content_encoded.columns if c.startswith("BestTimeToVisit_")]
    time_vec = pd.Series(0.0, index=time_cols)

    # MultiLabel de Preferences
    pref_cols = [c for c in destination_content_encoded.columns if c.startswith("Pref_")]
    pref_vec = pd.Series(0.0, index=pref_cols)
    for pref in user_profile["Preferences"].split(", "):
        col = f"Pref_{pref.strip()}"
        if col in pref_vec.index:
            pref_vec[col] = 1.0

    # Popularidad media
    popularity_mean = destination_content_encoded["PopularityNorm"].mean()

    # Concatenar
    user_vector = pd.concat([
        pd.Series({"PopularityNorm": popularity_mean}),
        type_vec,
        time_vec,
        pref_vec
    ]).to_frame().T

    # Ordenar columnas igual que en destino
    user_vector = user_vector[destination_content_encoded.drop(columns=["DestinationID", "Name_y"]).columns]

    # Convertir a float
    user_vector = user_vector.astype(float)

    # Similitud
    sim = cosine_similarity(
        destination_content_encoded.drop(columns=["DestinationID", "Name_y"]),
        user_vector
    ).flatten()

    recommendations = destination_content_encoded[["DestinationID", "Name_y"]].copy()
    recommendations["Similarity"] = sim
    recommendations = recommendations.sort_values("Similarity", ascending=False).head(top_n)
    recommendations = recommendations.rename(columns={"Name_y": "DestinationName"})
    return recommendations


In [322]:
# Recomendación basada en perfil
user_profile = {
    "Preferences": "Nature, Adventure",
    "Gender": "Female",
    "NumberOfAdults": 2,
    "NumberOfChildren": 1
}
recs_profile = recommend_by_profile(user_profile)
print("\n=== Recomendaciones basadas en perfil ===")
print(recs_profile)



=== Recomendaciones basadas en perfil ===
     DestinationID    DestinationName  Similarity
860          987.0        Goa Beaches    0.746210
517          468.0        Jaipur City    0.746188
31           114.0  Kerala Backwaters    0.746169
207          684.0  Kerala Backwaters    0.746167
84           130.0         Leh Ladakh    0.746157


In [323]:
import random

# -------------------------------
# Generador automático de perfiles
# -------------------------------
def generate_random_profiles(n_profiles=50):
    possible_preferences = [
        "Nature", "Adventure", "Culture", "Relaxation", "Beach", "Gastronomy"
    ]
    genders = ["Female", "Male"]
    num_adults_options = [1, 2, 3]
    num_children_options = [0, 1, 2]

    profiles = []
    for _ in range(n_profiles):
        selected_prefs = random.sample(possible_preferences, k=random.randint(1,2))
        profile = {
            "Preferences": ", ".join(selected_prefs),
            "Gender": random.choice(genders),
            "NumberOfAdults": random.choice(num_adults_options),
            "NumberOfChildren": random.choice(num_children_options)
        }
        profiles.append(profile)
    
    return profiles

# -------------------------------
# Evaluación contra múltiples usuarios
# -------------------------------
def evaluate_profile_recommendations_against_multiple_users(user_profile, recommended_df, user_ids):
    recommended_names = set(recommended_df["DestinationName"].values)

    precisions = []
    recalls = []

    for uid in user_ids:
        actual_destinations = set(df_clean[df_clean["UserID"] == uid]["Name_y"].unique())

        if not actual_destinations:
            continue  # Saltar usuarios sin historial

        hits = recommended_names.intersection(actual_destinations)

        precision = len(hits) / len(recommended_names) if recommended_names else 0
        recall = len(hits) / len(actual_destinations) if actual_destinations else 0

        precisions.append(precision)
        recalls.append(recall)

    avg_precision = sum(precisions) / len(precisions) if precisions else None
    avg_recall = sum(recalls) / len(recalls) if recalls else None

    return avg_precision, avg_recall

# -------------------------------
# Generar perfiles de prueba
# -------------------------------
test_profiles = generate_random_profiles(n_profiles=50)

# -------------------------------
# Evaluar todos los perfiles
# -------------------------------
all_user_ids = df_clean["UserID"].unique()

all_precisions = []
all_recalls = []

for idx, profile in enumerate(test_profiles):
    # Generar recomendaciones para este perfil
    recs_profile = recommend_by_profile(profile)

    # Evaluar contra todos los usuarios reales
    avg_precision, avg_recall = evaluate_profile_recommendations_against_multiple_users(
        profile, recs_profile, all_user_ids
    )

    all_precisions.append(avg_precision)
    all_recalls.append(avg_recall)

    print(f"Perfil {idx+1}: Precision={avg_precision:.3f}, Recall={avg_recall:.3f}")

# -------------------------------
# Métricas promedio globales
# -------------------------------
overall_precision = sum(all_precisions) / len(all_precisions)
overall_recall = sum(all_recalls) / len(all_recalls)

print("\n=== Métricas de evaluación global (promedio sobre perfiles aleatorios) ===")
print(f"Precision promedio global: {overall_precision:.3f}")
print(f"Recall promedio global: {overall_recall:.3f}")


Perfil 1: Precision=0.278, Recall=0.591
Perfil 2: Precision=0.278, Recall=0.591
Perfil 3: Precision=0.282, Recall=0.802
Perfil 4: Precision=0.282, Recall=0.802
Perfil 5: Precision=0.278, Recall=0.591
Perfil 6: Precision=0.282, Recall=0.802
Perfil 7: Precision=0.282, Recall=0.802
Perfil 8: Precision=0.278, Recall=0.591
Perfil 9: Precision=0.282, Recall=0.802
Perfil 10: Precision=0.282, Recall=0.802
Perfil 11: Precision=0.282, Recall=0.802
Perfil 12: Precision=0.278, Recall=0.591
Perfil 13: Precision=0.278, Recall=0.591
Perfil 14: Precision=0.282, Recall=0.802
Perfil 15: Precision=0.282, Recall=0.802
Perfil 16: Precision=0.278, Recall=0.591
Perfil 17: Precision=0.282, Recall=0.802
Perfil 18: Precision=0.282, Recall=0.802
Perfil 19: Precision=0.282, Recall=0.802
Perfil 20: Precision=0.278, Recall=0.591
Perfil 21: Precision=0.278, Recall=0.591
Perfil 22: Precision=0.282, Recall=0.802
Perfil 23: Precision=0.282, Recall=0.802
Perfil 24: Precision=0.278, Recall=0.591
Perfil 25: Precision=0.28

In [324]:
from sklearn.metrics import f1_score
from itertools import combinations

# ----------------------------
# 1) F1-Score promedio global
# ----------------------------

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

all_f1_scores = [compute_f1(p, r) for p, r in zip(all_precisions, all_recalls)]
overall_f1 = sum(all_f1_scores) / len(all_f1_scores)

print(f"\nF1-Score promedio global: {overall_f1:.3f}")

# ----------------------------
# 2) Hit Rate@5
# ----------------------------
# Proporción de perfiles donde al menos un destino recomendado coincide con destinos realmente visitados

hit_counts = 0
total_profiles = 0

for profile in test_profiles:
    recs_profile = recommend_by_profile(profile)
    recommended_names = set(recs_profile["DestinationName"].values)

    hits_any_user = False
    for uid in all_user_ids:
        actual_destinations = set(df_clean[df_clean["UserID"] == uid]["Name_y"].unique())
        if actual_destinations and recommended_names & actual_destinations:
            hits_any_user = True
            break

    if hits_any_user:
        hit_counts += 1
    total_profiles += 1

hit_rate = hit_counts / total_profiles
print(f"Hit Rate@5: {hit_rate:.3f}")

# ----------------------------
# 3) Cobertura de catálogo
# ----------------------------
# Proporción de destinos únicos que aparecen al menos una vez en todas las recomendaciones

all_recommended_destinations = set()

for profile in test_profiles:
    recs_profile = recommend_by_profile(profile)
    all_recommended_destinations.update(recs_profile["DestinationName"].values)

total_possible_destinations = df_clean["Name_y"].nunique()
catalog_coverage = len(all_recommended_destinations) / total_possible_destinations

print(f"Cobertura de catálogo: {catalog_coverage:.3f}")

# ----------------------------
# 4) Diversidad intra-lista promedio
# ----------------------------
# Para cada recomendación de perfil, medir cuán distintos son los destinos entre sí
# usando 1 - similitud del coseno de contenido

diversity_scores = []

# Precompute matrix of pairwise similarities
content_matrix = destination_content_encoded.drop(columns=["DestinationID", "Name_y"]).values
pairwise_content_sim = cosine_similarity(content_matrix)

# Mapping destino -> índice en matriz de contenido
dest_name_to_idx = dict(
    zip(destination_content_encoded["Name_y"], range(len(destination_content_encoded)))
)

for profile in test_profiles:
    recs_profile = recommend_by_profile(profile)
    names = recs_profile["DestinationName"].values

    if len(names) < 2:
        continue

    similarities = []
    for name1, name2 in combinations(names, 2):
        idx1 = dest_name_to_idx[name1]
        idx2 = dest_name_to_idx[name2]
        sim = pairwise_content_sim[idx1, idx2]
        similarities.append(sim)

    avg_similarity = sum(similarities) / len(similarities)
    diversity = 1 - avg_similarity
    diversity_scores.append(diversity)

avg_diversity = sum(diversity_scores) / len(diversity_scores)
print(f"Diversidad intra-lista promedio: {avg_diversity:.3f}")

# ----------------------------
# 5) Personalización promedio entre perfiles
# ----------------------------
# Compara cuán diferentes son las listas de recomendaciones entre pares de perfiles

pairwise_differences = []

recommendations_per_profile = []

for profile in test_profiles:
    recs_profile = recommend_by_profile(profile)
    recommendations_per_profile.append(set(recs_profile["DestinationName"].values))

for i, j in combinations(range(len(recommendations_per_profile)), 2):
    set_i = recommendations_per_profile[i]
    set_j = recommendations_per_profile[j]
    if not set_i or not set_j:
        continue

    intersection = set_i & set_j
    union = set_i | set_j

    jaccard_similarity = len(intersection) / len(union)
    personalization = 1 - jaccard_similarity
    pairwise_differences.append(personalization)

avg_personalization = sum(pairwise_differences) / len(pairwise_differences)
print(f"Personalización promedio entre perfiles: {avg_personalization:.3f}")



F1-Score promedio global: 0.401
Hit Rate@5: 1.000
Cobertura de catálogo: 0.800
Diversidad intra-lista promedio: 0.703
Personalización promedio entre perfiles: 0.124
